In [1]:
import importlib
import torch

from dataLoader.DataLoader import get_data_loader
from utils.Constants import tokenizer

from models.QABert import QABertTrainer
from train.Trainer import train_epoch

In [2]:
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'


Failed to find GPU. Will use CPU.


In [3]:
import dataLoader.DataLoader
import dataLoader.DataLoaderUtils


importlib.reload(dataLoader.DataLoader)
importlib.reload(dataLoader.DataLoaderUtils)

from dataLoader.DataLoader import get_data_loader
from dataLoader.DataLoaderUtils import get_question_answers_for_where_value_def_length, get_question_answers_def_length

dev_data_loader = get_data_loader(data_type = 'dev', tokenizer = tokenizer, batch_size = 1)

100%|██████████| 8421/8421 [00:42<00:00, 196.45it/s]


We have 8421 dev data with 2716 tables.


In [19]:
import utils.Constants
import models.WhereRanker

importlib.reload(models.WhereRanker)
importlib.reload(utils.Constants)

from models.WhereRanker import WhereRankerTrainer

where_ranker = WhereRankerTrainer(device, dev_data_loader)

iterator = iter(dev_data_loader)
for i in range(5):
    d = next(iterator)

input_ids = d["input_ids"].to(device)
attention_mask = d["attention_mask"].to(device)
token_type_ids = d["token_type_ids"].to(device)

agg_target = d["target"]['WHERE'].to(device)

agg_output = where_ranker.predict(
    input_ids,
    attention_mask,
    token_type_ids
)

where_ranker.calc_loss(
    agg_output, agg_target
)

KeyboardInterrupt: 

In [23]:
import models.WhereConditionClassifier

importlib.reload(models.WhereConditionClassifier)

from models.WhereConditionClassifier import WhereConditionClassifierTrainer

where_ranker = WhereConditionClassifierTrainer(device, dev_data_loader)

iterator = iter(dev_data_loader)
for i in range(5):
    d = next(iterator)

where_cond_targets = d["target"]['WHERE_CONDITIONS'].to(device)
where_columns = d["target"]['WHERE'].to(device)
num_where_columns = torch.count_nonzero(where_columns).item()
target_idx = torch.topk(where_columns, k=num_where_columns, dim=1)[1].to(device)

for where_column, where_cond_target in zip(target_idx.view(-1), where_cond_targets.view(-1)):
    where_outputs = where_ranker.predict(
        input_ids = input_ids,
        attention_mask = attention_mask,
        token_type_ids = token_type_ids,
        where_column = where_column
    )

    where_ranker.calc_loss(where_outputs, where_cond_target)


In [27]:
import models.QABert

importlib.reload(models.QABert)

from models.QABert import QABertTrainer

qa_ranker = QABertTrainer(device, dev_data_loader)

iterator = iter(dev_data_loader)
for i in range(5):
    d = next(iterator)

where_input_ids = d["qa_input_ids"].to(device)
where_attention_mask = d["qa_attention_mask"].to(device)
where_token_type_ids = d["qa_token_type_ids"].to(device)
for cond_num, where_cond_target in enumerate(d["target"]['WHERE_VALUE']):
    start_softmax, end_softmax = qa_ranker.predict(
        input_ids = where_input_ids.squeeze(0)[cond_num].view(-1),
        attention_mask = where_attention_mask.squeeze(0)[cond_num].view(-1),
        token_type_ids = where_token_type_ids.squeeze(0)[cond_num].view(-1),
    )

    qa_ranker.calc_loss(start_softmax, end_softmax, where_cond_target)


In [10]:
import models.SelectRanker
import models.WhereRanker
import train.Trainer
import models.AggregationClassifier
import models.WhereConditionClassifier
import models.WhereNumberClassifier
import utils.Constants

importlib.reload(models.QABert)
importlib.reload(models.SelectRanker)
importlib.reload(models.WhereRanker)
importlib.reload(models.WhereConditionClassifier)
importlib.reload(models.WhereNumberClassifier)
importlib.reload(models.AggregationClassifier)
importlib.reload(train.Trainer)
importlib.reload(utils.Constants)

from models.QABert import QABertTrainer
from models.SelectRanker import SelectRankerTrainer
from models.WhereRanker import WhereRankerTrainer
from models.WhereConditionClassifier import WhereConditionClassifierTrainer
from models.WhereNumberClassifier import WhereNumberClassifierTrainer
from models.AggregationClassifier import AggregationClassifierTrainer
from train.Trainer import train_epoch, save_model, load_model

models = dict(
    selection_trainer = SelectRankerTrainer(device, dev_data_loader),
    agg_class_trainer = AggregationClassifierTrainer(device, dev_data_loader),
    where_ranker_trainer = WhereRankerTrainer(device, dev_data_loader),
    where_cond_class_trainer = WhereConditionClassifierTrainer(device, dev_data_loader),
    where_numb_class_trainer = WhereNumberClassifierTrainer(device, dev_data_loader),
    qa_trainer = QABertTrainer(device, dev_data_loader),
)

#save_model(models,"./checkpoint")
#models = load_model("./checkpoint/16_Mar_2021_19_21", dev_data_loader, device)
train_epoch(models, dev_data_loader, device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

RuntimeError: 1D target tensor expected, multi-target not supported